In [29]:
import requests
import requests_cache
import lxml.html as lx
import re
from selenium import webdriver
import time
import random
import nltk

Testing to find format (for project report)

In [13]:
response= requests.get("https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc")
html_tree = lx.fromstring(response.text) #not necessairly working might need to use api

reviews = html_tree.xpath("//div[@id='reviews']//ul[@class = ' list__09f24__ynIEd']/li//div[@class = ' y-css-1pnalxe']/p/span/text()")

In [6]:
nextbuttons = html_tree.xpath("//span[@class = ' y-css-yrt0i5']//a[@class = 'pagination-link-component__09f24__JRiQO y-css-14k1rqv']/@href")
nextbuttons 

['https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=10',
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=20',
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=30',
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=40',
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=50',
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=60',
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=70',
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=80']

In [18]:
html_tree.xpath("(//a[div[@class='undefined y-css-1wz9c5l']])[last()]/div/@aria-label")


['Page: 9']

In [15]:
reviewlist = []
counter = 0 

while counter < (len(nextbuttons)):
    response = requests.get("https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc")
    html_tree = lx.fromstring(response.text)
    reviews = html_tree.xpath("//div[@id='reviews']//ul[@class = ' list__09f24__ynIEd']/li//div[@class = ' y-css-1pnalxe']/p/span/text()")
    reviewlist.extend(reviews)
    nextbuttons = html_tree.xpath("(//span[@class = ' y-css-yrt0i5']//a[@class = 'pagination-link-component__09f24__JRiQO y-css-14k1rqv'])/@href")
    for link in nextbuttons:
        if response.status_code != 200:
            print(f"Failed to fetch page: {link}")
            break
        else:
                response = requests.get(link)
                html_tree = lx.fromstring(response.text)
                reviews = html_tree.xpath("//div[@id='reviews']//ul[@class = ' list__09f24__ynIEd']/li//div[@class = ' y-css-1pnalxe']/p/span/text()")
                reviewlist.extend(reviews)
                counter+=1
                sleep_time = random.uniform(2, 5)  # Random delay to reduce likelihood of detection
                time.sleep(sleep_time)

Failed to fetch page: https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=40
Failed to fetch page: https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=30
Failed to fetch page: https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=20
Failed to fetch page: https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc&start=30


In [16]:
len(reviewlist)

151

Clean Code

In [23]:
def GetReviews(url):
    reviewlist = [] #creates a list of all reviews for given url
    visited_links = set() # no duplicates allowed
    nextbuttons = [url] #intializes parent url

    while nextbuttons:
        link = nextbuttons.pop(0) #adds to front to check
        if link in visited_links: #assures (with weird button format) that no html is parsed twice
            continue
        visited_links.add(link)

        response = requests.get(link) #request
        if response.status_code != 200:
            time.sleep(random.uniform(2, 5))
            response = requests.get(link)
            if response.status_code != 200:
                time.sleep(random.uniform(5, 10)) #longer time sleep if failed
                response = requests.get(link)
                if response.status_code != 200:
                        print(f"Failed to fetch page: {link}") #last resort
                        continue

        html_tree = lx.fromstring(response.text) 
        reviews = html_tree.xpath("//div[@id='reviews']//ul/li//p/span/text()") #paragraph container of reviews
        reviewlist.extend(reviews) #extends list

        # find next page links
        otherbuttons = html_tree.xpath("//a[contains(@class, 'pagination-link-component')]/@href") #other page links
        for button in otherbuttons:
            if button not in visited_links:
                nextbuttons.append(button) #checks next button for every link to assure every page of reviews is parsed

        # additional random delay to avoid detection
        time.sleep(random.uniform(2, 5))

    return reviewlist

In [18]:
urlist = ["https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/muni-bart-station-montgomery-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-muni-powell-street-station-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/muni-bart-station-civic-center-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-24th-st-mission-station-san-francisco-2?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-glen-park-station-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-balboa-park-station-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-west-oakland-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-12th-street-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-19th-street-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-macarthur-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-rockridge-station-oakland-2?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-fruitvale-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-coliseum-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-lake-merritt-station-oakland?sort_by=date_desc"]

In [79]:
#GetReviews("https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc")
#function works

In [76]:
reviewdict = {} #creating dict with url as key and list of reviews as values
for url in urlist:
    reviewdict[url] = GetReviews(url)

In [84]:
filtered_reviews = {
    url: [review for review in review_list if len(review.split()) > 2] #to make sure paragraphed reviews are longer than two words
    for url, review_list in reviewdict.items()
}
filtered_reviews

In [85]:
total_values = sum(len(value) if isinstance(value, list) else 0 for value in filtered_reviews.values())
print("There are " + str(total_values) + " reviews") #400 reviews filtered out

There are 5185 reviews


In [65]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [86]:
sid = SentimentIntensityAnalyzer()

def average_compound_score(reviews):
    """calculates the average compound sentiment score for a list of reviews."""
    compound_scores = []
    for sentence in reviews:
        ss = sid.polarity_scores(sentence)
        compound_scores.append(ss['compound'])
    return sum(compound_scores) / len(compound_scores) if compound_scores else 0

for key, reviews in filtered_reviews.items():  #goes through every review, uses nltk's VADER logic to compute and output sentimet score [-1,1]
    print(f"\nReviews for {key}:")
    for i, sentence in enumerate(reviews, start=1):
        print(f"  Review {i}: {sentence}")
        ss = sid.polarity_scores(sentence)
        for k in sorted(ss):
            print(f'    {k}: {ss[k]:.2f}', end=', ')
        print()  # newline after scores


Reviews for https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc:
  Review 1: This station has always had a problem with the homeless and it feels like the issues at this station has got worse over the years and both muni and Bart aren't doing much to solve it or to make it any better. If you're going through this station be careful.
    compound: 0.25,     neg: 0.10,     neu: 0.75,     pos: 0.15, 
  Review 2: Even though this is one of my favorite stations, I have to give it 1 star because I saw at least two people evade the toll on the way in. They did not have to pay today so I can't give it any more than one. Be better BART!
    compound: 0.76,     neg: 0.00,     neu: 0.85,     pos: 0.15, 
  Review 3: This is the first station coming into San Francisco from the East Bay, and probably the one I frequent the most now that I no longer work in The City.  
    compound: -0.35,     neg: 0.08,     neu: 0.92,     pos: 0.00, 
  Review 4: It is safe, well p

In [87]:
average_scores_dict = {}

# Iterate through the reviewdict and calculate average compound score for each station
for key, reviews in reviewdict.items():
    avg_compound = average_compound_score(reviews)
    average_scores_dict[key] = avg_compound

average_scores_dict

{'https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc': 0.07969885321100917,
 'https://www.yelp.com/biz/muni-bart-station-montgomery-san-francisco?sort_by=date_desc': 0.1785062937062937,
 'https://www.yelp.com/biz/bart-muni-powell-street-station-san-francisco?sort_by=date_desc': 0.15909181818181806,
 'https://www.yelp.com/biz/muni-bart-station-civic-center-san-francisco?sort_by=date_desc': 0.10631692094313455,
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc': 0.059416452442159366,
 'https://www.yelp.com/biz/bart-24th-st-mission-station-san-francisco-2?sort_by=date_desc': 0.09729007936507936,
 'https://www.yelp.com/biz/bart-glen-park-station-san-francisco?sort_by=date_desc': 0,
 'https://www.yelp.com/biz/bart-balboa-park-station-san-francisco?sort_by=date_desc': 0.14595945945945943,
 'https://www.yelp.com/biz/bart-west-oakland-station-oakland?sort_by=date_desc': 0.17753916913946577,
 'https://www.yelp.com/biz/bart-12th-s